In order to run this notebook, the following libaries must be installed:
... NumPy, Matplotlib, Scipy, Astropy, ArtPop, and Tqdm

# Interactive Color-Color Diagrams
Using ArtPop simple stellar populations (SSPs), this notebook produces an interactive color-color diagram for a set of SSPs. Points on the CCDs represent individual stellar populations, and can be clicked to reveal a plot of the selected population's color-magnitude diagram.

First, importing our needed libraries:

In [ ]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backend_bases import MouseButton
import scipy.spatial
from astropy import units as u
from astropy.io import ascii
from astropy.io import fits
import artpop
from artpop import MISTSSP
from tqdm.notebook import trange, tqdm
import pickle
%matplotlib tk

# Artpop's matplotlib style
plt.style.use(artpop.mpl_style)

# Random state for reproducibility
rng = np.random.RandomState(100)

Next, we define some helper functions:

In [ ]:
def from_pickle(file_name):
    """Load pickle of stellar population object."""
    pkl_file = open(file_name, 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    return data

global zpt_data
zpt_path = ''
zpt_data = ascii.read(zpt_path)

def zpt_convert_ab_to_vega(ab, filter):
    row = zpt_data['\ufefffilter' == filter]
    conversion = row['mag(Vega/AB)']
    return ab - conversion

def zpt_convert(ab, conversion):
    return ab - conversion

def log_mass(ssp):
    mass_string = str(ssp.total_initial_live_mass).replace('solMass', '')
    log_mass = np.log10(float(mass_string))
    return log_mass

We then use our from_pickle function to load in our list of SSPs. It's important that they are stored with the naming conventions used in the "SSP Library" notebook.

In [ ]:
num_ssps = 1000                 # Number of SSPs to be retrieved
dir = ''                        # Set your directory here

ssp_list = []                   # List storing SSP objects
ages = np.empty(num_ssps)       # Array storing age of each SSP
masses = np.empty(num_ssps)     # Array storing mass of each SSP
fehs = np.empty(num_ssps)       # Array storing metallicity of each SSP

for i in trange(num_ssps):
    fn = dir + '/MISTSSP' + str(i+1) + '.pickle'
    ssp_list.append(from_pickle(fn))
    ages[i] = ssp_list[i].log_age
    masses[i] = log_mass(ssp_list[i])
    fehs[i] = ssp_list[i].feh

After loading in our SSPs, we'll define some more helper functions:

In [ ]:
def on_click(event):
    # get the x and y pixel coords
    if event.button is MouseButton.LEFT:
        x, y = event.x, event.y
        if event.inaxes:
            ax = event.inaxes  # the axes instance
            #print('Data Coords:', (closest_point_coords(ckdtree, event.xdata, event.ydata)))
            #print('Index:', (closest_point_id(ckdtree, event.xdata, event.ydata)))
            i = closest_point_id(ckdtree, event.xdata, event.ydata)
            cmd(ssp_list[i], i)
            plt.disconnect(binding_id)

def cmd(ssp, i):
    # Generates a CMD based on selection from interactive plot
    f275w_mags, f336w_mags, f438w_mags, f555w_mags, f814w_mags = get_mags(ssp)

    #uv = f336w_mags - f438w_mags
    #nuv = f275w_mags - f438w_mags

    x = f555w_mags - f814w_mags
    y = f814w_mags

    plt.figure('Color-Magnitude Diagram for Cluster {}'.format(i))
    plt.scatter(x, y, c=x, cmap='Spectral_r')
    #cbar = plt.colorbar(orientation='horizontal')
    #cbar.ax.set_ylabel('Color')
    plt.title('CMD for Cluster {}'.format(i))
    plt.suptitle('Log[Age]: {}'.format(round(ssp.log_age, 3)),
                 ha='center', va='top')
    plt.suptitle('Log[Mass]: {}'.format(round(log_mass(ssp), 3)),
                 ha='center', va='top')
    plt.xlabel('Color (F555W - F814W)')
    plt.ylabel('Absolute Magnitude (F814W)')
    plt.gca().invert_yaxis()
    #plt.ylim([4, -12])
    #plt.xlim([-1, 4])

def closest_point_id(ckdtree, x, y):
    # returns index of closest point
    return ckdtree.query([x, y])[1]

def closest_point_coords(ckdtree, x, y):
    # returns coordinates of closest point
    return ckdtree.data[closest_point_id(ckdtree, x, y)]
    # ckdtree.data is the same as points

def val_shower(ckdtree):
    # formatter of coordinates displayed on Navigation Bar
    return lambda x, y: '[x = {}, y = {}]'.format(*closest_point_coords(ckdtree, x, y))

def get_mags_list(ssp_list):
    # Returns dictionary containing magnitudes of a given SSP in given filters
    n = len(ssp_list)

    f275w_mags = np.empty(n)
    f336w_mags = np.empty(n)
    f438w_mags = np.empty(n)
    f555w_mags = np.empty(n)
    f814w_mags = np.empty(n)

    for i in trange(n, desc='HST'):
        f275w_mags[i] = zpt_convert_ab_to_vega(ssp_list[i].isochrone.ssp_mag('WFC3_UVIS_F275W'), 'WFC3_UVIS_F275W')
        f336w_mags[i] = zpt_convert_ab_to_vega(ssp_list[i].isochrone.ssp_mag('WFC3_UVIS_F336W'), 'WFC3_UVIS_F336W')
        f438w_mags[i] = zpt_convert_ab_to_vega(ssp_list[i].isochrone.ssp_mag('WFC3_UVIS_F438W'), 'WFC3_UVIS_F438W')
        f555w_mags[i] = zpt_convert_ab_to_vega(ssp_list[i].isochrone.ssp_mag('WFC3_UVIS_F555W'), 'WFC3_UVIS_F555W')
        f814w_mags[i] = zpt_convert_ab_to_vega(ssp_list[i].isochrone.ssp_mag('WFC3_UVIS_F814W'), 'WFC3_UVIS_F814W')

    return f275w_mags, f336w_mags, f438w_mags, f555w_mags, f814w_mags

def get_mags(ssp):
    f275w_mags = zpt_convert_ab_to_vega(ssp.mag_table['WFC3_UVIS_F275W'], 'WFC3_UVIS_F275W')
    f336w_mags = zpt_convert_ab_to_vega(ssp.mag_table['WFC3_UVIS_F336W'], 'WFC3_UVIS_F336W')
    f438w_mags = zpt_convert_ab_to_vega(ssp.mag_table['WFC3_UVIS_F438W'], 'WFC3_UVIS_F438W')
    f555w_mags = zpt_convert_ab_to_vega(ssp.mag_table['WFC3_UVIS_F555W'], 'WFC3_UVIS_F555W')
    f814w_mags = zpt_convert_ab_to_vega(ssp.mag_table['WFC3_UVIS_F814W'], 'WFC3_UVIS_F814W')

    return f275w_mags, f336w_mags, f438w_mags, f555w_mags, f814w_mags

Next, we can use our get_mags function to obtain our desired magnitude lists, and plot them on a color-color diagram:

In [ ]:
f275w_mags, f336w_mags, f438w_mags, f555w_mags, f814w_mags = get_mags_list(ssp_list)

In [ ]:
u_b = f336w_mags - f438w_mags
nuv_b = f275w_mags - f438w_mags
v_i = f555w_mags - f814w_mags

points = np.column_stack([v_i, u_b])
ckdtree = scipy.spatial.cKDTree(points)

plt.figure('Color-Color Diagram of All Clusters')
plt.scatter(points[:,0], points[:,1], c=ages, s=2**masses*5, cmap='magma') # mass-determined marker size
binding_id = plt.connect('motion_notify_event', on_click)
plt.title('HST U-B vs V-I')
plt.connect('button_press_event', on_click)
plt.xlabel('F555W - F814W')
plt.ylabel('F336W - F438W')
cbar = plt.colorbar()
cbar.ax.set_ylabel('log[age]')
cbar.ax.invert_yaxis()
plt.gca().invert_yaxis()
plt.gca().format_coord = val_shower(ckdtree)